Docs:
- [IntStream.forEach](https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#forEach-java.util.function.IntConsumer-)
- [IntStream.peek](https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#peek-java.util.function.IntConsumer-)
- [ArrayList.forEach](https://docs.oracle.com/javase/8/docs/api/java/util/ArrayList.html#forEach-java.util.function.Consumer-)
- [the Consumer interface](https://docs.oracle.com/javase/8/docs/api/java/util/function/Consumer.html)
- [the Function interface](https://docs.oracle.com/javase/8/docs/api/java/util/function/Function.html)
- [the Comparator interface](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Comparator.html)
- [Stream.map](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html#map-java.util.function.Function-)
- [List.sort](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/List.html#sort(java.util.Comparator))

In [30]:
var arange = IntStream.range(0, 10)

java.util.stream.IntPipeline$Head@489b1d29

In [31]:
var arangeList = arange.boxed().collect(Collectors.toList()); // converting the IntStream to a normal (boxed) stream, then converting it to a List
arangeList;

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [36]:
Consumer fn = (x) -> System.out.println(x);
arangeList.forEach(fn); 

0
1
2
3
4
5
6
7
8
9


null

Type inference will be done automatically if the lambda expression is used directly in a method call (as the compiler knows the type signature needed):

In [37]:
arangeList.forEach((x) -> System.out.println(x)); 

0
1
2
3
4
5
6
7
8
9


null

But the compiler is not smart enough for this kind of type inference:

In [38]:
var fn = (x) -> System.out.println(x);
arange.forEach(fn);

var fn = (x) -> System.out.println(x);: var fn = (x) -> System.out.println(x);

It's possible to use the general type `Function`, but then it might not be compatible with the functional interface needed by the caller:

In [43]:
Function fn = (x) -> { 
    System.out.println(x);
    return null;
    };

arangeList.forEach(fn);

: 

Now let's see a pure function that is used for computation instead of side-effects.

We first have to convert our `List` to a `Stream`, then convert it back into a `List`. This is because Java's `List` does not have a `map` method.

In [59]:
arangeList.stream().map((x) -> {
   return x*10; 
}).collect(Collectors.toList());

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

We can use the generic type `Function` for these; but we will still lose some type information. This causes us to need to cast `x` to `int` explicitly.

In [65]:
Function fn = (x) -> {
    var x_int = (int)x;
    return x_int*10;
};

arangeList.stream().map(fn).collect(Collectors.toList());

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Let's define a helper function to run `map` on `List` more concisely:

In [68]:
List listMap(List lst, Function fn) {
    return (List)(lst.stream().map(fn).collect(Collectors.toList()));
}

null

In [69]:
Function fn = (x) -> {
    var x_int = (int)x;
    return x_int*10;
};

listMap(arangeList, fn);

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

# Stream functions

In [106]:
var arange = IntStream.range(0, 10); // We need to recreate this `stream` every time we use it, as it will be "consumed."

arange
  .filter((x) -> (x % 2 == 0))
  .toArray();

[0, 2, 4, 6, 8]

In [110]:
var arange = IntStream.range(0, 10); 
arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .toArray();

[2, 4, 6, 8]

In [113]:
var arange = IntStream.range(0, 10); 
arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .toArray();

[6, 12, 18, 24]

In [121]:
var arange = IntStream.range(0, 10); 

arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .reduce(0, (a, b) -> a+b); // https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#reduce-int-java.util.function.IntBinaryOperator-

60

We could also use the function reference `Integer::sum` for the last reduction:

In [123]:
Integer.sum(8,11); // an example

19

In [122]:
var arange = IntStream.range(0, 10); 

arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .reduce(0, Integer::sum);

60

# Function composition

## Composing functions returning Void (i.e., running functions for their side-effects)

In [48]:
Consumer consumer_sequential_combination(List<Consumer> consumers) {
    return (x) -> {
        consumers.forEach((consumer) -> consumer.accept(x)); 
        // We looked at the docs to know that this interface uses the name `accept` for its "main" method.
    };
}

null

In [54]:
Consumer fn_print = (x) -> System.out.println(x);
Consumer fn_double_print = (x) -> {
    int x_int = (int)x;
    System.out.println(x_int*2);
};
Consumer fn_print_sep = (x) -> System.out.println("#####");

Consumer fn_all = consumer_sequential_combination(Arrays.asList(fn_print, fn_double_print, fn_print_sep));

arangeList.forEach(fn_all); 

0
0
#####
1
2
#####
2
4
#####
3
6
#####
4
8
#####
5
10
#####
6
12
#####
7
14
#####
8
16
#####
9
18
#####


null

Exercise: Create `consumer_parallel_combination` that runs its consumers in parallel.

## Composing pure functions returning results

In [80]:
Function fn_compose(List<Function> functions) {
    return (x) -> {
        Object res = x;
        for (Function fn : functions) {
            res = fn.apply(res);
            // https://docs.oracle.com/javase/8/docs/api/java/util/function/Function.html
        }
        return res;
    };
}

null

In [90]:
Function fn_double = (x) -> {
    var x_int = (int)x;
    return x_int * 10;
};

Function fn_inc = (x) -> {
    var x_int = (int)x;
    return x_int + 1;
};

Function fn_all = fn_compose(Arrays.asList(fn_double, fn_inc, fn_inc, fn_inc));

listMap(arangeList, fn_all);

[3, 13, 23, 33, 43, 53, 63, 73, 83, 93]

# Using lambdas for sorting

In [100]:
arangeList.sort((a, b) -> {
   if (a == 3) {
       return -1; // '3' is smaller than everything else.
   } else if (b == 3) {
       return 1; // everything is bigger than '3'.
   } else if (a < b) {
       return -1;
   } else if (a == b) {
       return 0;
   } else {
       return 1;
   }
});

arangeList;

[3, 0, 1, 2, 4, 5, 6, 7, 8, 9]

We can use the following helper function [Comparator.comparingInt](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Comparator.html#comparingInt(java.util.function.ToIntFunction)) which makes the code cleaner.

`static Comparator`   `comparingInt​(ToIntFunction<? super T> keyExtractor)`:   Accepts a function that extracts an `int` sort key from a type `T`, and returns a `Comparator` that compares by that sort key.

In [103]:
arangeList.sort(Comparator.comparingInt((x) -> {
    if (x == 3) {
        return Integer.MIN_VALUE;
    }
    
    return x;
}));

arangeList;

[3, 0, 1, 2, 4, 5, 6, 7, 8, 9]

# Tmp

In [17]:
IntConsumer fn = (x) -> System.out.println(x);
arange.peek(fn); 

java.util.stream.IntPipeline$10@49cea0df

In [19]:
arange.peek((x) -> System.out.println(x)); 

null